<a href="https://www.kaggle.com/code/sarah1999/nlp-lab3-part2?scriptVersionId=197684338" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install -U nltk==3.4

     |████████████████████████████████| 1.4 MB 3.5 MB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for nltk: filename=nltk-3.4-py3-none-any.whl size=1436396 sha256=fb811e7aefa08de40829c0c0e9d148e49b3df9b10de15b291f6e5e9eb6216566
  Stored in directory: /root/.cache/pip/wheels/13/b8/81/2349be11dd144dc7b68ab983b58cd2fae353cdc50bbdeb09d0
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.4 which is incompatible.


# Language model on tweets

In [2]:
import pandas as pd
import numpy as np
import regex as re
import nltk
import random
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize import TweetTokenizer
import emoji
from nltk.lm import MLE


In [3]:
data = pd.read_csv("../input/large-random-tweets-from-pakistan/Random "
                   "Tweets from Pakistan- Cleaned- Anonymous.csv",encoding_errors = 'ignore')

data = data['full_text']
data = data.dropna()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
data.head()

0    تیرا لیڈر میرا لیڈر نواز شریف نواز شریف❤️  کیا...
1    Happy birthday to my brother n boss , May you ...
2                                                 ❤️❤️
3    `suspicious °jikook au jimin'in yaşadığı kasab...
4    Speaking of @SpiderMan... 😂   https://t.co/uGJ...
Name: full_text, dtype: object

### Preprocessing and tokenization

In [5]:
from nltk.tokenize import TweetTokenizer
import re

# Initialize the TweetTokenizer
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

# Sample tweet before preprocessing
print('Tweet before preprocessing and tokenization: \n', data[10])

# Preprocessing steps

# Ensure all data is treated as strings before applying regex
data = data.apply(lambda x: str(x))

# removing @Mentions
data = data.apply(lambda x: re.sub(r'@\w+', '', x))

# removing #hashtags
data = data.apply(lambda x: re.sub(r'#\w+', '', x))

# removing RT
data = data.apply(lambda x: re.sub(r'RT[\s]+', '', x))

# removing websites (URLs)
data = data.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))

# removing extra symbols (if any specific symbols are to be removed, this can be adjusted)
data = data.apply(lambda x: re.sub(r'[^\w\s]', '', x))  # keeping only alphanumeric characters and spaces

# removing extra spaces
data = data.apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Tokenizing text
data = data.apply(tokenizer.tokenize)

# Sample tweet after preprocessing and tokenization
print('Tweet after preprocessing and tokenization: \n', data[10])


Tweet before preprocessing and tokenization: 
 Cant sleep 🥵🥱 So why not . https://t.co/WjmRMBOSBC
Tweet after preprocessing and tokenization: 
 ['cant', 'sleep', 'so', 'why', 'not']


In [6]:
# Preprocess the tokenized text for 2-grams language modelling
n_gram_size = 3  #  bigrams
train_data, padded_sents = padded_everygram_pipeline(n_gram_size, data)


In [7]:
# Define the MLE model (ensure n_gram_size is properly defined)
model = MLE(n_gram_size)

# Fit the model using the training data
model.fit(train_data, padded_sents)

# Ensure the vocabulary is properly initialized
print(model.vocab)
print(f"Vocabulary size: {len(model.vocab)}")

# Testing model on Out Of Vocabulary words (OOV) or unknown tokens
print(model.vocab.lookup(['here', 'is', 'a', 'solution', 'for', 'it']))
print(model.vocab.lookup(['cant', 'find', 'blabla', 'and', 'blablabla']))


<Vocabulary with cutoff=1 unk_label='<UNK>' and 146500 items>
Vocabulary size: 146500
('here', 'is', 'a', 'solution', 'for', 'it')
('cant', 'find', '<UNK>', 'and', '<UNK>')


In [8]:
# Total number of 2-grams
print("",model.counts)

 <NgramCounter with 3 ngram orders and 10916943 ngrams>


In [9]:
# frequencies of occurences
print("Count of unigram 'pakistan': ",model.counts['pakistan'])
print("Count of bigram 'is/pakistan': ",model.counts[['pakistan']]['is'])
print("Count of trigram 'a/pakistan is': ",model.counts[['pakistan', 'is']]['a'])

Count of unigram 'pakistan':  12720
Count of bigram 'is/pakistan':  689
Count of trigram 'a/pakistan is':  70


In [10]:
# probablities of occurences
print("probability of 'pakistan' given <s>: ",model.score('pakistan'))
print("probability of 'is' given 'pakistan': ",model.score('is',['pakistan']))
print("probability of 'a' given 'pakistan is': ",model.score('a',['pakistan','is']))

probability of 'pakistan' given <s>:  0.0033115237903826268
probability of 'is' given 'pakistan':  0.05416666666666667
probability of 'a' given 'pakistan is':  0.10159651669085631


In [11]:
# Unigram count of 'pakistan'
print("Count of unigram 'pakistan': ", model.counts['pakistan'])

# Bigram count of 'is/pakistan'
print("Count of bigram 'pakistan is': ", model.counts[('pakistan',)]['is'])

# Trigram count of 'a/pakistan is'
print("Count of trigram 'pakistan is a': ", model.counts[('pakistan', 'is')]['a'])

# Probabilities
# Probability of 'pakistan' given start of sentence <s>
print("Probability of 'pakistan' given <s>: ", model.score('pakistan', ['<s>']))

# Probability of 'is' given 'pakistan'
print("Probability of 'is' given 'pakistan': ", model.score('is', ['pakistan']))

# Probability of 'a' given 'pakistan is'
print("Probability of 'a' given 'pakistan is': ", model.score('a', ['pakistan', 'is']))


Count of unigram 'pakistan':  12720
Count of bigram 'pakistan is':  689
Count of trigram 'pakistan is a':  70
Probability of 'pakistan' given <s>:  0.003054647268625928
Probability of 'is' given 'pakistan':  0.05416666666666667
Probability of 'a' given 'pakistan is':  0.10159651669085631


In [12]:
# Perplexity of our model on sample text
model.perplexity('pakistan')

1303.5488228387676